# Pandas 2

Analizaremos características de precio y superficie, por barrio y por comuna, de inmuebles en venta en la Ciudad de Buenos Aires en el año 2016.


## Dataset

El Gobierno de la Ciudad de Buenos Aires disponibiliza algunos datasets para acceso público en la url 
https://data.buenosaires.gob.ar/

Allí encontramos datos de propiedades inmuebles en venta, separados por año
https://data.buenosaires.gob.ar/dataset/departamentos-venta

En esta clase vamos a usar un subconjunto de los datos de "Departamentos en venta 2016" que descargamos desde aquí
https://data.buenosaires.gob.ar/dataset/departamentos-venta/archivo/juqdkmgo-7031-resource

En la carpeta Data de esta clase ya tenemos descargardo ese dataset, el nombre del archivo es **departamentos-en-venta-2016.csv**.

## Imports

In [2]:
import pandas as pd
import numpy as np

## Ejercicio 1  - Importar 

Leamos los datos del archivo departamentos-en-venta-2016.csv

Veamos cuántos registros tiene y de qué tipos son los datos de cada columna. 

Veamos los primeros registros para verificar que los datos fueron importados correctamente.

In [3]:
import chardet
def get_encoding_type(csv_path):
    rawdata = open(csv_path, 'rb').read()
    result = chardet.detect(rawdata)
    return result.get('encoding')

In [4]:
data_location = "../Data/departamentos-en-venta-2016.csv"

In [5]:
## [BORRAR_PRESENCIAL]
file_encoding = get_encoding_type(data_location)
file_encoding

'utf-8'

In [6]:
## [BORRAR_PRESENCIAL]
data = pd.read_csv(data_location, sep=";", encoding=file_encoding)
data.shape

(7564, 29)

In [7]:
data.dtypes

CALLE                       object
NUMERO                     float64
ID_ZONAPRO                   int64
OPERACION                   object
TIPO                        object
M2                           int64
M2CUB                        int64
PRECIOTEXT                  object
PRECIOARS                    int64
PRECIOARSM                   int64
DOLARES                      int64
U_S_M2                       int64
AMBIENTES                    int64
ANTIGUEDAD                   int64
BAÑOS                        int64
DIRECCION                   object
LOCATION                    object
PUBLICADO                   object
PROCESADO                   object
URL                         object
REVISION                   float64
NOTA                       float64
DIRECCION_NORMALIZADA       object
BARRIO                      object
COMUNA                      object
CODIGO_POSTAL              float64
CODIGO_POSTAL_ARGENTINO     object
LATITUD                    float64
LONGITUD            

In [8]:
data.head()

,CALLE,NUMERO,ID_ZONAPRO,OPERACION,TIPO,M2,M2CUB,PRECIOTEXT,PRECIOARS,PRECIOARSM,...,URL,REVISION,NOTA,DIRECCION_NORMALIZADA,BARRIO,COMUNA,CODIGO_POSTAL,CODIGO_POSTAL_ARGENTINO,LATITUD,LONGITUD
0,GUATEMALA,5574.0,42408691,VTA,DTO,57,50,U$S 170.150,2977625,59553,...,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42408...,NaN,NaN,GUATEMALA 5574,PALERMO,COMUNA 14,1425.0,C1425BVH,-34.580581,-58.431758
1,ZAPATA,300.0,42408710,VTA,DTO,46,46,U$S 118.650,2076375,45139,...,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42408...,NaN,NaN,ZAPATA 300,PALERMO,COMUNA 14,1426.0,C1426AED,-34.573870,-58.440609
2,ZAPATA,300.0,42518390,VTA,DTO,61,56,U$S 181.470,3175725,56709,...,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42518...,NaN,NaN,ZAPATA 300,PALERMO,COMUNA 14,1426.0,C1426AED,-34.573870,-58.440609
3,ZAPATA,300.0,42518402,VTA,DTO,140,76,U$S 320.000,5600000,73684,...,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42518...,NaN,NaN,ZAPATA 300,PALERMO,COMUNA 14,1426.0,C1426AED,-34.573870,-58.440609
4,"JUSTO, JUAN B. AV.",2300.0,42621693,VTA,DTO,39,33,U$S 82.116,1437030,43546,...,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42621...,NaN,NaN,"JUSTO, JUAN B. AV. 2300",VILLA CRESPO,COMUNA 15,1414.0,C1414CWY,-34.592984,-58.441542


## Ejercicio 2  - Promedio

¿Cuál es el promedio valor del precio por $m^2$ en dolares?

Para responder esta pregunta vamos a:

1) Crear un obeto Series que resulte de limpiar los valores del campo PRECIOTEXT. Tenemos que eliminar los símbolos "U\\$S" y "."

2) En la instancia de Series resultado del punto 1) reemplazamos los valores en pesos ('\\$') por nulos (para que nos ensucien los datos de promedio). 

3) Crear una columna nueva de tipo numérico en el DataFrame (PRECIOTEXT_CLEAN) y asignar los valores de resultado de 2)

3) Crear una columna de tipo numérico (PRECIOTEXTM) que tenga el precio del m2 en dolares calculado como el valor de PRECIOTEXT_CLEAN / M2

4) Calcular la media de los valores de PRECIOTEXTM


In [9]:
import re

# reemplazo U$S por vacío
dolares_pattern = "U\$S\s*"
dolares_regex = re.compile(dolares_pattern)

# reemplazo el punto decimal por vacío
decimal_pattern = "\."
decimal_regex = re.compile(decimal_pattern)

sin_dolar = data.PRECIOTEXT.apply(lambda x: x if x is np.NaN else dolares_regex.sub("", x))

sin_dolar_sin_punto = sin_dolar.apply(lambda x: x if x is np.NaN else decimal_regex.sub("", x))

#type(sin_dolar_sin_punto)

In [10]:
# armo una máscara con los registros que tienen el valor de PRECIOTEXT en $
pesos_pattern = "\$"
pesos_regex = re.compile(pesos_pattern)

matches_pesos = sin_dolar_sin_punto.apply(lambda x: x if x is np.NaN else pesos_regex.match(x))
pesos_mask = matches_pesos.notnull()
#pesos_mask.sum()

#veo que egfectivamente esos son los valores con $
print(sin_dolar_sin_punto[pesos_mask])

# asigno np.NaN a esos valores
sin_dolar_sin_punto[pesos_mask] = np.NaN


142        $13000
156     $ 1140000
257        $89000
278     $ 1396000
279     $ 1927000
          ...    
6556    $ 1021734
6557    $ 1188007
6558    $ 1093950
6559    $ 1077120
6560    $ 1108868
Name: PRECIOTEXT, Length: 308, dtype: object


In [11]:
sin_dolar_sin_punto_sin_pesos_num  = sin_dolar_sin_punto.astype(float)
data["PRECIOTEXT_CLEAN"] = sin_dolar_sin_punto_sin_pesos_num
#print(data.dtypes)
data["PRECIOTEXTM"] = data["PRECIOTEXT_CLEAN"] / data["M2"]
#data.head(3)
precio_promedio_m2 = data["PRECIOTEXTM"].mean()
precio_promedio_m2.round(2)

2522.49

## Ejercicio 3  - Promedio por cuartil
¿Cuál es el precio promedio del metro cuadrado en dolares para cada cuartil de superficie (campo M2) de las viviendas en CABA?

Comenzamos calculando los cuartilos de superficie.

In [12]:
q_superficie = pd.qcut(data.M2, 4)
q_superficie

0         (54.0, 80.0]
1         (41.0, 54.0]
2         (54.0, 80.0]
3        (80.0, 730.0]
4       (14.999, 41.0]
             ...      
7559    (14.999, 41.0]
7560      (41.0, 54.0]
7561      (41.0, 54.0]
7562      (41.0, 54.0]
7563      (41.0, 54.0]
Name: M2, Length: 7564, dtype: category
Categories (4, interval[float64]): [(14.999, 41.0] < (41.0, 54.0] < (54.0, 80.0] < (80.0, 730.0]]

In [13]:
data.groupby(q_superficie)["PRECIOTEXTM"].mean().round(2)

M2
(14.999, 41.0]    2418.49
(41.0, 54.0]      2455.99
(54.0, 80.0]      2438.61
(80.0, 730.0]     2788.30
Name: PRECIOTEXTM, dtype: float64

## Ejercicio 3  - Promedio por barrio

¿Cuál es la media de precio por metro cuadrado en dolares para cada barrio de la CABA? 

Ordenemos los datos para indicar cuál es el barrio más caro.

Resolvamos el cálculo tanto con groupby como con pivot tables

In [14]:
data.groupby('BARRIO')['PRECIOTEXTM'].mean().round(2).sort_values(ascending = False)

BARRIO
PUERTO MADERO        5412.47
PALERMO              3136.01
BELGRANO             3110.07
RETIRO               2941.05
RECOLETA             2876.20
PARQUE CHACABUCO     2645.41
COLEGIALES           2636.72
NUÑEZ                2597.47
COGHLAN              2451.12
CABALLITO            2412.78
VILLA ORTUZAR        2408.77
VILLA URQUIZA        2402.51
SAAVEDRA             2365.06
MONSERRAT            2349.13
CHACARITA            2338.94
VILLA CRESPO         2331.95
AGRONOMIA            2276.70
ALMAGRO              2252.27
VILLA DEL PARQUE     2240.72
BOEDO                2215.39
CONSTITUCION         2197.24
VILLA DEVOTO         2190.42
SAN NICOLAS          2186.05
BARRACAS             2166.15
VILLA PUEYRREDON     2159.10
SAN TELMO            2145.82
PARQUE CHAS          2137.48
BALVANERA            1988.28
BOCA                 1963.58
VILLA LURO           1960.14
MONTE CASTRO         1952.63
PARQUE PATRICIOS     1943.39
VILLA GRAL. MITRE    1928.58
VERSALLES            1926.17
SAN CRI

In [15]:
data.pivot_table(index = 'BARRIO', aggfunc={'PRECIOTEXTM':'mean'}) \
                    . round(2) \
                    .sort_values(by='PRECIOTEXTM',ascending=False)

,PRECIOTEXTM
BARRIO,
PUERTO MADERO,5412.47
PALERMO,3136.01
BELGRANO,3110.07
RETIRO,2941.05
RECOLETA,2876.20
PARQUE CHACABUCO,2645.41
COLEGIALES,2636.72
NUÑEZ,2597.47
COGHLAN,2451.12


## Ejercicio 4  - Dispersión de precios por barrio

**4.a** ¿En qué barrio hay una mayor dispersión en el valor del metro cuadrado en dolares? Ordenar los valores para identificar el mayor.

Ayuda: Calculamos dispersión como el desvío estandar de un grupo dividido la media de ese grupo grupo

Vamos a calcular esto de dos maneras distintas:

1) Calculamos la instancia de Series que tiene la media por grupo. Calculamos la instancia de Series que tiene el desvío estandar por grupo. Las dividimos. Ordenamos

2) Calculamos los grupos y usamos una función lambda que calcule la media, desvío estandar y divida. Ordenamos.

**4.b** Cuál les parece que será más eficiente? Probemoslo con %timeit, para eso definamos dos funciones que encapsulen el código de 1) y 2).

**4.c** ¿Por qué Paternal está devolviendo NaN? ¿Y por qué Villa Soldati devuelve 0?

Nota: tengan en cuenta que `mean` y `std` sobre una instancia de `DataFrameGroupBy` excluye los valores nulos

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.mean.html

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.std.html





In [16]:
def dispersion_1(data):
    barrio_group = data.groupby('BARRIO')
    medias_barrio = barrio_group['PRECIOTEXTM'].mean()
    desvios_barrio = barrio_group['PRECIOTEXTM'].std()
    dipersion = desvios_barrio / medias_barrio
    result = dipersion.sort_values(ascending=False) 
    return result

def dispersion_2(data):
    result = data.groupby('BARRIO')['PRECIOTEXTM'].apply(lambda x: x.std() / x.mean()).sort_values(ascending = False)
    return result
    

In [17]:
dispersion_1(data)

BARRIO
VILLA DEVOTO         0.440995
BOCA                 0.425274
MONSERRAT            0.321376
PALERMO              0.308486
BALVANERA            0.300993
NUÑEZ                0.295307
VELEZ SARSFIELD      0.275630
RECOLETA             0.263813
CHACARITA            0.249468
SAN TELMO            0.248734
FLORES               0.248200
BELGRANO             0.245323
RETIRO               0.244298
PUERTO MADERO        0.234363
MATADEROS            0.229643
VILLA LURO           0.227212
COLEGIALES           0.217556
BARRACAS             0.212347
SAAVEDRA             0.212152
CONSTITUCION         0.208357
LINIERS              0.198838
ALMAGRO              0.191560
MONTE CASTRO         0.188504
VILLA URQUIZA        0.185133
VILLA CRESPO         0.182353
VILLA DEL PARQUE     0.180029
SAN CRISTOBAL        0.176112
SAN NICOLAS          0.170378
VILLA GRAL. MITRE    0.168801
CABALLITO            0.167364
COGHLAN              0.164022
BOEDO                0.150507
VILLA PUEYRREDON     0.149338
PAR

In [18]:
dispersion_2(data)

BARRIO
VILLA DEVOTO         0.440995
BOCA                 0.425274
MONSERRAT            0.321376
PALERMO              0.308486
BALVANERA            0.300993
NUÑEZ                0.295307
VELEZ SARSFIELD      0.275630
RECOLETA             0.263813
CHACARITA            0.249468
SAN TELMO            0.248734
FLORES               0.248200
BELGRANO             0.245323
RETIRO               0.244298
PUERTO MADERO        0.234363
MATADEROS            0.229643
VILLA LURO           0.227212
COLEGIALES           0.217556
BARRACAS             0.212347
SAAVEDRA             0.212152
CONSTITUCION         0.208357
LINIERS              0.198838
ALMAGRO              0.191560
MONTE CASTRO         0.188504
VILLA URQUIZA        0.185133
VILLA CRESPO         0.182353
VILLA DEL PARQUE     0.180029
SAN CRISTOBAL        0.176112
SAN NICOLAS          0.170378
VILLA GRAL. MITRE    0.168801
CABALLITO            0.167364
COGHLAN              0.164022
BOEDO                0.150507
VILLA PUEYRREDON     0.149338
PAR

In [19]:
%timeit dispersion_1(data)

2.33 ms ± 74.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
%timeit dispersion_2(data)

16.4 ms ± 107 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Vamos a ver cuánto da si usamos la lambda directamente, sin pasar por dispersion_2:

In [21]:
%timeit data.groupby('BARRIO')['PRECIOTEXTM'].apply(lambda x: x.std() / x.mean()).sort_values(ascending = False)

16.6 ms ± 333 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Veamos por qué Paternal devuelve NaN y Villa Soldati que devuelve 0.

Para eso vamos a contar cuántos registros tenemos en cada barrio, en particular en Paternal.

In [22]:
barrio_count = data.groupby('BARRIO')['BARRIO'].count()
print(barrio_count["PATERNAL"])

1


Hay un sólo registro en el barrio de Paternal, entonces el desvío está dividiendo por 0 y por eso devuelve NaN

In [23]:
print(barrio_count["VILLA SOLDATI"])

villa_soldati_mask = data.BARRIO == "VILLA SOLDATI"
villa_soldati_data = data.loc[villa_soldati_mask, :]
villa_soldati_data["PRECIOTEXTM"]

2


3059    812.5
3060    812.5
Name: PRECIOTEXTM, dtype: float64

Vemos que para Villa Soldati los dos registros tienen valores iguales, entonces el numerador en el cálculo del desvío estandar da 0, y por lo tanto también el desvío estandar y la dispersión resultan 0.


## Ejercicio 5  - Superficie por barrio

Calculemos la mediana de superficie por barrio para determinar en qué barrio los departamentos son más grandes.

Hagamos el cálculo con pivot_table y groupby


In [24]:
data.pivot_table('M2', index='BARRIO', aggfunc=np.median).sort_values(by='M2', ascending=False)

,M2
BARRIO,
PUERTO MADERO,117.0
VILLA SOLDATI,80.0
PATERNAL,77.0
MATADEROS,75.0
COGHLAN,74.5
BELGRANO,74.0
RECOLETA,74.0
VELEZ SARSFIELD,73.0
FLORES,71.0


In [25]:
data.groupby('BARRIO')['M2'].median().sort_values(ascending=False)

BARRIO
PUERTO MADERO        117.0
VILLA SOLDATI         80.0
PATERNAL              77.0
MATADEROS             75.0
COGHLAN               74.5
BELGRANO              74.0
RECOLETA              74.0
VELEZ SARSFIELD       73.0
FLORES                71.0
VERSALLES             70.5
VILLA DEL PARQUE      70.5
COLEGIALES            70.0
RETIRO                68.0
BARRACAS              65.5
VILLA LURO            62.0
PARQUE CHAS           61.5
VILLA LUGANO          60.0
NUÑEZ                 60.0
BOCA                  59.5
LINIERS               58.5
PALERMO               57.0
PARQUE PATRICIOS      56.0
FLORESTA              56.0
VILLA DEVOTO          55.5
BALVANERA             55.0
SAAVEDRA              54.5
CABALLITO             54.0
SAN TELMO             53.0
VILLA ORTUZAR         53.0
ALMAGRO               53.0
VILLA URQUIZA         52.0
SAN CRISTOBAL         52.0
CONSTITUCION          50.0
VILLA PUEYRREDON      48.0
MONTE CASTRO          47.0
BOEDO                 47.0
VILLA REAL           

## Ejercicio 6  - Tabla resumen

Generemos un `DataFrame` que agregue la información del precio por M2 en dolares (`PRECIOTEXTM`), ambientes (`AMBIENTES`) a nivel de `COMUNA` y barrio (`BARRIO`). Proporcione información tanto de la tendencia central como de la dispersión de ambas distribuciones.

Ayuda: usar pivot_table

In [26]:
result = data.pivot_table(['PRECIOTEXTM','AMBIENTES'],index=['COMUNA','BARRIO']
               , aggfunc={'PRECIOTEXTM':[np.mean,np.std,len],
                          'AMBIENTES': [np.mean,np.std]})
print(type(result))
result

<class 'pandas.core.frame.DataFrame'>


AMBIENTES           PRECIOTEXTM               \
                                 mean       std         len         mean   
COMUNA    BARRIO                                                           
COMUNA 01 CONSTITUCION       2.208696  0.893403       115.0  2197.241584   
          MONSERRAT          2.100000  1.087115       100.0  2349.127916   
          PUERTO MADERO      2.678571  0.765320        56.0  5412.468738   
          RETIRO             2.892473  1.684084        93.0  2941.052687   
          SAN NICOLAS        2.190698  1.232696       215.0  2186.049883   
          SAN TELMO          2.256410  0.594623        39.0  2145.817227   
COMUNA 02 RECOLETA           2.931204  1.353770       407.0  2876.201544   
COMUNA 03 BALVANERA          2.476636  1.046755       214.0  1988.284935   
          SAN CRISTOBAL      2.397869  0.621211       563.0  1914.203956   
COMUNA 04 BARRACAS           2.583333  1.088918        84.0  2166.146707   
          BOCA               2.312500  1.138347        16.0  1963.575157   
          NUEVA POMPEYA      2.000000  0.000000         2.0  1622.727273   
          PARQUE PATRICIOS   2.208955  0.640485        67.0  1943.392373   
COMUNA 05 ALMAGRO            2.370149  1.094490       335.0  2252.269793   
          BOEDO              2.032258  1.224306        31.0  2215.391462   
COMUNA 06 CABALLITO          2.528771  0.947383       643.0  2412.777566   
COMUNA 07 FLORES             2.806452  0.875488        93.0  1905.204204   
          PARQUE CHACABUCO   2.330579  0.541969       363.0  2645.412539   
COMUNA 08 VILLA LUGANO       2.333333  1.211060         6.0  1138.726815   
          VILLA SOLDATI      4.000000  0.000000         2.0   812.500000   
COMUNA 09 LINIERS            2.290323  0.662511        62.0  1885.041244   
          MATADEROS          2.727273  0.674200        33.0  1804.158874   
          PARQUE AVELLANEDA  2.000000  0.000000         5.0  1806.324289   
COMUNA 10 FLORESTA           2.277778  1.017815        18.0  1711.078346   
          MONTE CASTRO       2.400000  0.709721        55.0  1952.634631   
          VELEZ SARSFIELD    3.200000  2.049390         5.0  1748.636059   
          VERSALLES          3.000000  0.000000         2.0  1926.168413   
          VILLA LURO         2.705882  1.046704        17.0  1960.143435   
          VILLA REAL         2.666667  0.577350         3.0  1738.200049   
COMUNA 11 VILLA DEL PARQUE   2.692308  1.319674        26.0  2240.717867   
          VILLA DEVOTO       2.653846  1.809802        26.0  2190.415302   
          VILLA GRAL. MITRE  2.285714  0.673024        42.0  1928.578367   
          VILLA SANTA RITA   2.137931  0.789400        29.0  1907.998258   
COMUNA 12 COGHLAN            2.684211  0.808912        38.0  2451.120924   
          SAAVEDRA           2.204819  1.000110       166.0  2365.061414   
          VILLA PUEYRREDON   2.000000  0.946864        30.0  2159.102203   
          VILLA URQUIZA      2.157609  1.009326       184.0  2402.505658   
COMUNA 13 BELGRANO           2.767442  1.125982       473.0  3110.065913   
          COLEGIALES         2.726316  1.046190        95.0  2636.723717   
          NUÑEZ              2.595745  0.999848       141.0  2597.472491   
COMUNA 14 PALERMO            2.491256  1.247419       629.0  3136.014984   
COMUNA 15 AGRONOMIA          1.833333  0.983192         6.0  2276.703779   
          CHACARITA          1.764706  1.087901        51.0  2338.939470   
          PARQUE CHAS        1.937500  0.573730        16.0  2137.475822   
          PATERNAL           3.000000       NaN         1.0  1720.779221   
          VILLA CRESPO       2.255639  0.838384       532.0  2331.951318   
          VILLA ORTUZAR      2.264151  0.593262        53.0  2408.765643   

                                          
                                     std  
COMUNA    BARRIO                          
COMUNA 01 CONSTITUCION        457.810983  
          MONSERRAT           754.954231  
          PU